In [100]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import StackingClassifier

In [101]:
train_df = pd.read_csv("output.csv")
test_df = pd.read_csv("test.csv")

In [102]:
train_df.head(3)

,id,Ложь,Аггравация,Экстраверсия,Спонтанность,Агрессивность,Ригидность,Интроверсия,Сензитивность,Тревожность,Лабильность,Эмоциональное_состояние_опрошенного,Результат
0,0,4.0,4.0,4.0,2.0,3.0,7.0,5.0,3.0,6.0,4.0,отлично,0
1,1,8.0,7.0,6.0,6.0,5.0,5.0,5.0,5.0,6.0,6.0,удовлетворительно,0
2,2,6.0,7.0,4.0,4.0,3.0,4.0,5.0,3.0,3.0,5.0,отлично,0


In [103]:
test_df.head(3)

,id,Ложь,Аггравация,Экстраверсия,Спонтанность,Агрессивность,Ригидность,Интроверсия,Сензитивность,Тревожность,Лабильность,Эмоциональное_состояние_опрошенного,Результат
0,0,3.0,2.0,3.0,2.0,4.0,3.0,2.0,9.0,8.0,6.0,отлично,NaN
1,1,2.0,3.0,2.0,4.0,3.0,4.0,3.0,7.0,6.0,5.0,отлично,NaN
2,2,4.0,4.0,6.0,5.0,6.0,7.0,8.0,9.0,8.0,7.0,удовлетворительно,NaN


In [104]:
cat_cols = [
    "Эмоциональное_состояние_опрошенного"]
num_cols = ['id','Ложь','Аггравация','Экстраверсия','Спонтанность','Агрессивность',
            'Ригидность','Интроверсия','Сензитивность','Тревожность','Лабильность']

In [105]:
# заполняем пробелы
for column in num_cols:
    train_df[column].fillna(train_df[column].mean(), inplace=True)
for column in cat_cols:
    train_df[column].fillna(train_df[column].mode()[0], inplace=True)

In [106]:
test_train_df = train_df.drop(cat_cols, axis=1)
test_train_df = test_train_df.drop(["id", "Результат"], axis=1)

In [107]:
matrix = test_train_df.corr().abs() # создаем матрицу корреляций

threshold = 0.7 # порог можно поставить любой,
                # для примера возьмем 0.7, однако, можно поэкспериментировать
                # с разными значениями

upper = matrix.where(np.triu(np.ones(matrix.shape), k=1).astype(np.bool_))
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
catboost_df = test_train_df.drop(to_drop, axis=1)
catboost_df.corr().style.background_gradient(cmap='coolwarm')

,Ложь,Аггравация,Экстраверсия,Спонтанность,Ригидность,Интроверсия,Сензитивность,Тревожность
Ложь,1.000000,0.459642,0.302121,-0.188234,-0.072126,-0.199862,-0.054469,0.042455
Аггравация,0.459642,1.000000,0.121281,-0.499439,-0.474600,-0.046803,0.061488,0.325556
Экстраверсия,0.302121,0.121281,1.000000,-0.339729,-0.461688,-0.699168,0.053104,-0.251790
Спонтанность,-0.188234,-0.499439,-0.339729,1.000000,0.450564,0.249429,0.315891,-0.101408
Ригидность,-0.072126,-0.474600,-0.461688,0.450564,1.000000,0.165930,-0.229829,0.148269
Интроверсия,-0.199862,-0.046803,-0.699168,0.249429,0.165930,1.000000,0.490409,0.508327
Сензитивность,-0.054469,0.061488,0.053104,0.315891,-0.229829,0.490409,1.000000,0.533720
Тревожность,0.042455,0.325556,-0.251790,-0.101408,0.148269,0.508327,0.533720,1.000000


In [108]:
# I prepared a function for reviewing the dataset.
def Df_info(df):
    print("########### Head ###########")
    print(df.head())
    print("########### Tail ###########")
    print(df.tail())
    print("########### Shape ###########")
    print(df.shape)
    print("########### Info ###########")
    print(df.info())
    print("########### Info ###########")
    print(df.columns)
    print("########### Quantiles ###########")
    print(df.describe().T)
    print("########### NA ###########")
    print(df.isnull().sum())

In [109]:
catboost_df[cat_cols] = train_df[cat_cols]

# Df_info(catboost_df)

In [110]:
Df_info(catboost_df)

########### Head ###########
   Ложь  Аггравация  Экстраверсия  Спонтанность  Ригидность  Интроверсия  \
0   4.0         4.0           4.0           2.0         7.0          5.0   
1   8.0         7.0           6.0           6.0         5.0          5.0   
2   6.0         7.0           4.0           4.0         4.0          5.0   
3   7.0         5.0           5.0           5.0         6.0          8.0   
4   4.0         4.0           7.0           5.0         5.0          4.0   

   Сензитивность  Тревожность Эмоциональное_состояние_опрошенного  
0            3.0          6.0                             отлично  
1            5.0          6.0                   удовлетворительно  
2            3.0          3.0                             отлично  
3            5.0          8.0                 неудовлетворительно  
4            3.0          3.0                             отлично  
########### Tail ###########
     Ложь  Аггравация  Экстраверсия  Спонтанность  Ригидность  Интроверсия  \

In [111]:
catboost_df.shape

(151, 9)

In [112]:
catboost_df["Результат"] = train_df["Результат"]

In [113]:
catboost_df

,Ложь,Аггравация,Экстраверсия,Спонтанность,Ригидность,Интроверсия,Сензитивность,Тревожность,Эмоциональное_состояние_опрошенного,Результат
0,4.0,4.0,4.0,2.0,7.0,5.0,3.0,6.0,отлично,0
1,8.0,7.0,6.0,6.0,5.0,5.0,5.0,6.0,удовлетворительно,0
2,6.0,7.0,4.0,4.0,4.0,5.0,3.0,3.0,отлично,0
3,7.0,5.0,5.0,5.0,6.0,8.0,5.0,8.0,неудовлетворительно,1
4,4.0,4.0,7.0,5.0,5.0,4.0,3.0,3.0,отлично,0
...,...,...,...,...,...,...,...,...,...,...
146,6.0,2.0,1.0,9.0,9.0,7.0,3.0,3.0,неудовлетворительно,1
147,3.0,2.0,5.0,6.0,4.0,7.0,5.0,3.0,удовлетворительно,1
148,5.0,6.0,6.0,4.0,6.0,6.0,4.0,4.0,отлично,0
149,1.0,4.0,1.0,9.0,7.0,8.0,5.0,7.0,неудовлетворительно,1


In [123]:
for column in num_cols:
    test_df[column].fillna(test_df[column].mean(), inplace=True)
for column in cat_cols[:-1]:
    test_df[column].fillna(test_df[column].mode()[0], inplace=True)

In [124]:
test_df = test_df[catboost_df.columns[:-1]]
test_df.shape

(10, 9)

In [125]:
test_df

,Ложь,Аггравация,Экстраверсия,Спонтанность,Ригидность,Интроверсия,Сензитивность,Тревожность,Эмоциональное_состояние_опрошенного
0,3.0,2.0,3.0,2.0,3.0,2.0,9.0,8.0,отлично
1,2.0,3.0,2.0,4.0,4.0,3.0,7.0,6.0,отлично
2,4.0,4.0,6.0,5.0,7.0,8.0,9.0,8.0,удовлетворительно
3,5.0,5.0,6.0,7.0,6.0,5.0,2.0,3.0,удовлетворительно
4,6.0,7.0,9.0,8.0,6.0,7.0,3.0,4.0,неудовлетворительно
5,7.0,8.0,9.0,8.0,6.0,5.0,2.0,3.0,неудовлетворительно
6,8.0,9.0,8.0,7.0,5.0,4.0,3.0,2.0,неудовлетворительно
7,4.0,3.0,2.0,5.0,3.0,6.0,7.0,8.0,удовлетворительно
8,3.0,4.0,5.0,6.0,6.0,5.0,4.0,3.0,удовлетворительно
9,5.0,6.0,7.0,8.0,8.0,7.0,6.0,8.0,неудовлетворительно


In [115]:
catboost_df[cat_cols] = catboost_df[cat_cols].astype("str")
test_df[cat_cols] = test_df[cat_cols].astype("str")

In [116]:
Df_info(catboost_df)

########### Head ###########
   Ложь  Аггравация  Экстраверсия  Спонтанность  Ригидность  Интроверсия  \
0   4.0         4.0           4.0           2.0         7.0          5.0   
1   8.0         7.0           6.0           6.0         5.0          5.0   
2   6.0         7.0           4.0           4.0         4.0          5.0   
3   7.0         5.0           5.0           5.0         6.0          8.0   
4   4.0         4.0           7.0           5.0         5.0          4.0   

   Сензитивность  Тревожность Эмоциональное_состояние_опрошенного  Результат  
0            3.0          6.0                             отлично          0  
1            5.0          6.0                   удовлетворительно          0  
2            3.0          3.0                             отлично          0  
3            5.0          8.0                 неудовлетворительно          1  
4            3.0          3.0                             отлично          0  
########### Tail ###########
     Ложь  

In [117]:
X = catboost_df.drop(["Результат"], axis=1)
y = catboost_df["Результат"]

x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

In [118]:
model = CatBoostClassifier(iterations=1000,
                           depth=6,
                           task_type="GPU",
                           devices='0:1')
model.fit(x_train, y_train, cat_features=cat_cols)

Learning rate set to 0.037881
0:	learn: 0.6148206	total: 41.2ms	remaining: 41.1s
1:	learn: 0.5481800	total: 72.7ms	remaining: 36.3s
2:	learn: 0.4905975	total: 139ms	remaining: 46s
3:	learn: 0.4344416	total: 172ms	remaining: 42.9s
4:	learn: 0.3853286	total: 199ms	remaining: 39.5s
5:	learn: 0.3424131	total: 234ms	remaining: 38.7s
6:	learn: 0.3023245	total: 263ms	remaining: 37.3s
7:	learn: 0.2718267	total: 303ms	remaining: 37.5s
8:	learn: 0.2461758	total: 340ms	remaining: 37.4s
9:	learn: 0.2206615	total: 369ms	remaining: 36.5s
10:	learn: 0.2001728	total: 412ms	remaining: 37s
11:	learn: 0.1814951	total: 452ms	remaining: 37.2s
12:	learn: 0.1645327	total: 496ms	remaining: 37.6s
13:	learn: 0.1503604	total: 541ms	remaining: 38.1s
14:	learn: 0.1364631	total: 580ms	remaining: 38.1s
15:	learn: 0.1247014	total: 643ms	remaining: 39.6s
16:	learn: 0.1137879	total: 683ms	remaining: 39.5s
17:	learn: 0.1041071	total: 708ms	remaining: 38.6s
18:	learn: 0.0963344	total: 746ms	remaining: 38.5s
19:	learn: 0.

In [119]:
y_pred_proba = model.predict_proba(x_val)
y_pred_proba.shape, y_val.shape

((31, 2), (31,))

In [120]:
roc_auc_score(y_val, y_pred_proba[:,1])

1.0

In [121]:
test_df.head(3)

,id,Ложь,Аггравация,Экстраверсия,Спонтанность,Агрессивность,Ригидность,Интроверсия,Сензитивность,Тревожность,Лабильность,Эмоциональное_состояние_опрошенного,Результат
0,0,3.0,2.0,3.0,2.0,4.0,3.0,2.0,9.0,8.0,6.0,отлично,NaN
1,1,2.0,3.0,2.0,4.0,3.0,4.0,3.0,7.0,6.0,5.0,отлично,NaN
2,2,4.0,4.0,6.0,5.0,6.0,7.0,8.0,9.0,8.0,7.0,удовлетворительно,NaN


In [126]:
test_pred_proba = model.predict_proba(test_df)
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=model.classes_)
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

In [127]:
test_pred_proba_df.shape

(10, 2)

In [128]:
test_pred_proba_df.head()

,0,1
0,0.846010,0.153990
1,0.617859,0.382141
2,0.000528,0.999472
3,0.986118,0.013882
4,0.226509,0.773491


In [129]:
test_pred_proba_df.to_csv("test_pred_proba_df.csv", index=False)